# Market to implement TI

### Business Context
A technology company located in Cundinamarca - Colombia has recently developed a new business line know as Analytics - AI, this beacuse the company has seen the change in the trend respect that companies not only need the engineer structure but also the real implementation of AI technologies into the companies. 

### Business Problem
The company is really new in this line, however based on its own history they know that once they develop a solution it is possible to offer it as a sector solution, in that way they want to know what is the most common types of industries and their location in the way to put their efforts in the most common sector, but also to make a business plan with the second and third common.

### Data requirements and description
1. 116 cities located in Cundinamarca - Colombia and its location (longitude and latitude).
    * Get information DANE resource.
    * Codigo_Depto: Code to identify each department.
    * Nombre_Depto: Name of each department.
    * Provincia: Group by each province where town is located inside each department.
    * Codigo_Municipio: Codes of each town.
    * Nombre_Municipio: Name of each town.
    * Latitud: Latitude of each town
    * Longitud: Longitude of each town
2. Companies located in Cundinamarca and its sector.

### Final Deliver
1. Create a 10 cluster of each 116 cities, to see easily what is the common line and similitudes inside each group. 

In [1]:
## import libraries we need to get clusters.
import pandas as pd
import numpy as np

# library for web scraping
import requests
from bs4 import BeautifulSoup
import re

from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [47]:
## Import the tables from xlsx file
Cundinamarca = pd.read_excel('C:/Users/gabrielamabec/Personal_Projects/Data_Science_Basics/Data Capstone/Departamentos.xlsx',
                             sheet_name = 'Cundinamarca')
Cundinamarca

,Codigo_Depto,Nombre_Depto,Provincia,Codigo_Municipio,Nombre_Municipio,Latitud,Longitud
0,25,Cundinamarca,Almeidas,25183,Chocontá,5.1447,-73.6858
1,25,Cundinamarca,Almeidas,25426,Machetá,5.0815,-73.6076
2,25,Cundinamarca,Almeidas,25436,Manta,5.0086,-73.5412
3,25,Cundinamarca,Almeidas,25736,Sesquilé,5.0446,-73.7972
4,25,Cundinamarca,Almeidas,25772,Suesca,5.1029,-73.7985
...,...,...,...,...,...,...,...
112,25,Cundinamarca,Ubate,25779,Susa,5.4519,-73.8144
113,25,Cundinamarca,Ubate,25781,Sutatausa,5.2478,-73.8524
114,25,Cundinamarca,Ubate,25793,Tausa,5.1990,-73.8913
115,25,Cundinamarca,Ubate,25843,Ubaté,5.3093,-73.8157


In [62]:
Group_Cun = Cundinamarca.groupby('Provincia').count()
Group_Cun

,Codigo_Depto,Nombre_Depto,Codigo_Municipio,Nombre_Municipio
Provincia,,,,
Almeidas,7,7,7,7
Alto Magdalena,8,8,8,8
Bajo Magdalena,3,3,3,3
Bogotá,1,1,1,1
Gualiva,12,12,12,12
Guavio,8,8,8,8
Magdalena Centro,7,7,7,7
Medina,2,2,2,2
Oriente,10,10,10,10


In [63]:
Group_Cun.describe()

,Codigo_Depto,Nombre_Depto,Codigo_Municipio,Nombre_Municipio
count,16.000000,16.000000,16.000000,16.000000
mean,7.312500,7.312500,7.312500,7.312500
std,3.439356,3.439356,3.439356,3.439356
min,1.000000,1.000000,1.000000,1.000000
25%,6.000000,6.000000,6.000000,6.000000
50%,8.000000,8.000000,8.000000,8.000000
75%,10.000000,10.000000,10.000000,10.000000
max,12.000000,12.000000,12.000000,12.000000


### Analysis:
1. The data set has 117 rows where 116 are towns and 1 is a city (Bogotá).
2. Gualiva is the provincia that has the biggest number of towns in Cundinamarca.
3. There are 5 provincias that are formed by 10 towns.
4. Normally provincias are formed by 8 or more towns.

In [3]:
# Define credentials for all the process to work with 
CLIENT_ID = 'Client ID' # your Foursquare ID
CLIENT_SECRET = 'Client Secret' # your Foursquare Secret
VERSION = 'Version API' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your are ready to work!')

Your are ready to work!


In [35]:
# Define a function to get information about near places of each postal codes
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Codigo_Municipio', 
                  'Codigo_Municipio Latitude', 
                  'Codigo_Municipio Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [48]:
# Get the information about each town based on latitude and longitude
LIMIT = 100
Cundinamarca_venues = getNearbyVenues(names=Cundinamarca['Codigo_Municipio'],
                                latitudes=Cundinamarca['Latitud'],
                                longitudes=Cundinamarca['Longitud'],
                                radius = 1000
                                )

25183
25426
25436
25736
25772
25807
25873
25001
25307
25324
25368
25483
25488
25612
25815
25148
25320
25572
25019
25398
25402
25489
25491
25592
25658
25718
25777
25851
25862
25875
25293
25297
25299
25322
25326
25372
25377
25839
25086
25095
25168
25328
25580
25662
25867
25438
25530
25151
25178
25181
25279
25281
25335
25339
25594
25841
25845
25258
25394
25513
25518
25653
25823
25871
25885
25126
25175
25200
25295
25486
25758
25785
25817
25899
25099
25214
25260
25269
25286
25430
25473
25769
25799
25898
25740
25754
25053
25120
25290
25312
25524
25535
25649
25743
25805
25506
25035
25040
25599
25123
25245
25386
25596
25645
25797
25878
25154
25224
25288
25317
25407
25745
25779
25781
25793
25843
11001


In [49]:
# Lets check the information we get about each postal code.
print(Cundinamarca_venues.shape)
Cundinamarca_venues.head()

(567, 7)


,Codigo_Municipio,Codigo_Municipio Latitude,Codigo_Municipio Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,25183,5.1447,-73.6858,Villa cecilia,5.141552,-73.691034,Restaurant
1,25183,5.1447,-73.6858,parador punto exacto,5.148967,-73.681595,Arepa Restaurant
2,25183,5.1447,-73.6858,Mis Antojitos,5.143401,-73.689934,Coffee Shop
3,25183,5.1447,-73.6858,Panaderia Y Cafeteria Los Cristales,5.143877,-73.689786,Breakfast Spot
4,25426,5.0815,-73.6076,Machetá,5.080417,-73.608607,City


In [53]:
# Before to understand information we get, first we combine information about location with venues
Cundinamarca = Cundinamarca[['Codigo_Depto','Nombre_Depto','Provincia','Codigo_Municipio','Nombre_Municipio']]
Cundinamarca_data = pd.merge(left = Cundinamarca_venues, right = Cundinamarca, on = 'Codigo_Municipio')
Cundinamarca_data.head()

,Codigo_Municipio,Codigo_Municipio Latitude,Codigo_Municipio Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Codigo_Depto,Nombre_Depto,Provincia,Nombre_Municipio
0,25183,5.1447,-73.6858,Villa cecilia,5.141552,-73.691034,Restaurant,25,Cundinamarca,Almeidas,Chocontá
1,25183,5.1447,-73.6858,parador punto exacto,5.148967,-73.681595,Arepa Restaurant,25,Cundinamarca,Almeidas,Chocontá
2,25183,5.1447,-73.6858,Mis Antojitos,5.143401,-73.689934,Coffee Shop,25,Cundinamarca,Almeidas,Chocontá
3,25183,5.1447,-73.6858,Panaderia Y Cafeteria Los Cristales,5.143877,-73.689786,Breakfast Spot,25,Cundinamarca,Almeidas,Chocontá
4,25426,5.0815,-73.6076,Machetá,5.080417,-73.608607,City,25,Cundinamarca,Almeidas,Machetá


In [59]:
# Create a groupby by provincia to see where we get more information.
Cundinamarca_data.groupby('Provincia').count()

,Codigo_Municipio,Codigo_Municipio Latitude,Codigo_Municipio Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Codigo_Depto,Nombre_Depto,Nombre_Municipio
Provincia,,,,,,,,,,
Almeidas,29,29,29,29,29,29,29,29,29,29
Alto Magdalena,24,24,24,24,24,24,24,24,24,24
Bajo Magdalena,10,10,10,10,10,10,10,10,10,10
Bogotá,100,100,100,100,100,100,100,100,100,100
Gualiva,32,32,32,32,32,32,32,32,32,32
Guavio,26,26,26,26,26,26,26,26,26,26
Magdalena Centro,7,7,7,7,7,7,7,7,7,7
Medina,3,3,3,3,3,3,3,3,3,3
Oriente,28,28,28,28,28,28,28,28,28,28


#### Analysis


In [64]:
# Create a one hot encoding data frame to look main characteristics
Cundinamarca_onehot = pd.get_dummies(Cundinamarca_data[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Cundinamarca_onehot['Nombre_Municipio'] = Cundinamarca_data['Nombre_Municipio'] 

# move neighborhood column to the first column
fixed_columns = [Cundinamarca_onehot.columns[-1]] + list(Cundinamarca_onehot.columns[:-1])
Cundinamarca_onehot = Cundinamarca_onehot[fixed_columns]

# Review the new data frame transformation
print(Cundinamarca_onehot.shape)
Cundinamarca_onehot.head()

(567, 134)


,Nombre_Municipio,Advertising Agency,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,Auto Workshop,...,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Restaurant,Tourist Information Center,Vegetarian / Vegan Restaurant,Volleyball Court,Wings Joint
0,Chocontá,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Chocontá,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Chocontá,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Chocontá,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Machetá,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [65]:
# Create a groupby by each Neighbourhood based on mean of each characteristics
Cundinamarca_grouped = Cundinamarca_onehot.groupby('Nombre_Municipio').mean().reset_index()
Cundinamarca_grouped.head()

,Nombre_Municipio,Advertising Agency,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,Auto Workshop,...,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Restaurant,Tourist Information Center,Vegetarian / Vegan Restaurant,Volleyball Court,Wings Joint
0,Agua De Dios,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Albán,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Anapoima,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Anolaima,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Apulo,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [66]:
# Review the top 5 common venues on each characteristics.
num_top_venues = 10

# Create a loop to see main characteristics.
for hood in Cundinamarca_grouped['Nombre_Municipio']:
    print("----"+hood+"----")
    temp = Cundinamarca_grouped[Cundinamarca_grouped['Nombre_Municipio'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agua De Dios----
                 venue  freq
0                Plaza   0.5
1        National Park   0.5
2   Advertising Agency   0.0
3             Mountain   0.0
4            Pool Hall   0.0
5                 Pool   0.0
6          Planetarium   0.0
7          Pizza Place   0.0
8             Pharmacy   0.0
9  Peruvian Restaurant   0.0


----Albán----
                 venue  freq
0           Restaurant   0.5
1                 Park   0.5
2   Advertising Agency   0.0
3             Mountain   0.0
4                 Pool   0.0
5                Plaza   0.0
6          Planetarium   0.0
7          Pizza Place   0.0
8             Pharmacy   0.0
9  Peruvian Restaurant   0.0


----Anapoima----
                       venue  freq
0              Big Box Store  0.14
1                  BBQ Joint  0.14
2       Colombian Restaurant  0.14
3          Convenience Store  0.14
4                      Plaza  0.14
5                Pizza Place  0.14
6                      Hotel  0.14
7  Latin American Restaura

                    venue  freq
0      Italian Restaurant  0.07
1           Shopping Mall  0.07
2          Breakfast Spot  0.07
3  Argentinian Restaurant  0.07
4                     Pub  0.07
5          Ice Cream Shop  0.07
6        Department Store  0.07
7           Burrito Place  0.07
8                    Food  0.07
9               BBQ Joint  0.07


----Nariño----
                  venue  freq
0       Harbor / Marina   1.0
1    Advertising Agency   0.0
2                  Park   0.0
3         Movie Theater   0.0
4             Multiplex   0.0
5         National Park   0.0
6       Nature Preserve   0.0
7          Optical Shop   0.0
8  Other Great Outdoors   0.0
9   Peruvian Restaurant   0.0


----Nemocón----
                  venue  freq
0                  City   0.2
1           Art Gallery   0.2
2  Caribbean Restaurant   0.2
3                 Plaza   0.2
4  Other Great Outdoors   0.2
5   Peruvian Restaurant   0.0
6         National Park   0.0
7       Nature Preserve   0.0
8          Op

                        venue  freq
0                        Park  0.25
1                       Hotel  0.25
2  Tourist Information Center  0.25
3                      Resort  0.25
4          Advertising Agency  0.00
5                   Multiplex  0.00
6               National Park  0.00
7             Nature Preserve  0.00
8                Optical Shop  0.00
9        Other Great Outdoors  0.00


----Supatá----
                 venue  freq
0                 Farm   1.0
1   Advertising Agency   0.0
2             Mountain   0.0
3                 Pool   0.0
4                Plaza   0.0
5          Planetarium   0.0
6          Pizza Place   0.0
7             Pharmacy   0.0
8  Peruvian Restaurant   0.0
9                 Park   0.0


----Susa----
                  venue  freq
0           Cheese Shop   1.0
1    Advertising Agency   0.0
2   Peruvian Restaurant   0.0
3             Multiplex   0.0
4         National Park   0.0
5       Nature Preserve   0.0
6          Optical Shop   0.0
7  Other Grea

9  Movie Theater  0.00




In [67]:
# Create a function with the most common venues
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [77]:
# Define the top of venues we want to see in the data frame.
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Nombre_Municipio']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
Cundinamarca_sorted = pd.DataFrame(columns=columns)
Cundinamarca_sorted['Nombre_Municipio'] = Cundinamarca_grouped['Nombre_Municipio']

for ind in np.arange(Cundinamarca_grouped.shape[0]):
    Cundinamarca_sorted.iloc[ind, 1:] = return_most_common_venues(Cundinamarca_grouped.iloc[ind, :], num_top_venues)

Cundinamarca_sorted.head()

,Nombre_Municipio,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agua De Dios,Plaza,National Park,Wings Joint,Farm,Clothing Store,Cocktail Bar,Coffee Shop,Colombian Restaurant,Convenience Store,Cupcake Shop
1,Albán,Park,Restaurant,Convenience Store,Diner,Dessert Shop,Department Store,Deli / Bodega,Dance Studio,Cupcake Shop,Wings Joint
2,Anapoima,Plaza,Hotel,BBQ Joint,Convenience Store,Colombian Restaurant,Big Box Store,Pizza Place,Coffee Shop,Cocktail Bar,Farmers Market
3,Anolaima,Salad Place,Fried Chicken Joint,Park,Convenience Store,Dessert Shop,Department Store,Deli / Bodega,Dance Studio,Cupcake Shop,Wings Joint
4,Apulo,Soccer Field,Gym / Fitness Center,Basketball Stadium,Park,Wings Joint,Dance Studio,Diner,Dessert Shop,Department Store,Deli / Bodega


In [75]:
# set number of clusters
kclusters = 10

Cundinamarca_grouped_clustering = Cundinamarca_grouped.drop('Nombre_Municipio', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Cundinamarca_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 1, 0, 1, 0, 1, 0, 0, 0, 1])

In [78]:
# add clustering labels
Cundinamarca_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Cundinamarca_merged = Cundinamarca_data

# merge toronto grouped with toronto_data to add latitude/longitude for each neighborhood
Cundinamarca_merged = Cundinamarca_merged.join(Cundinamarca_sorted.set_index('Nombre_Municipio'), on='Nombre_Municipio')

Cundinamarca_merged.head()

,Codigo_Municipio,Codigo_Municipio Latitude,Codigo_Municipio Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Codigo_Depto,Nombre_Depto,Provincia,...,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,25183,5.1447,-73.6858,Villa cecilia,5.141552,-73.691034,Restaurant,25,Cundinamarca,Almeidas,...,Arepa Restaurant,Breakfast Spot,Coffee Shop,Restaurant,Wings Joint,Deli / Bodega,Eastern European Restaurant,Diner,Dessert Shop,Department Store
1,25183,5.1447,-73.6858,parador punto exacto,5.148967,-73.681595,Arepa Restaurant,25,Cundinamarca,Almeidas,...,Arepa Restaurant,Breakfast Spot,Coffee Shop,Restaurant,Wings Joint,Deli / Bodega,Eastern European Restaurant,Diner,Dessert Shop,Department Store
2,25183,5.1447,-73.6858,Mis Antojitos,5.143401,-73.689934,Coffee Shop,25,Cundinamarca,Almeidas,...,Arepa Restaurant,Breakfast Spot,Coffee Shop,Restaurant,Wings Joint,Deli / Bodega,Eastern European Restaurant,Diner,Dessert Shop,Department Store
3,25183,5.1447,-73.6858,Panaderia Y Cafeteria Los Cristales,5.143877,-73.689786,Breakfast Spot,25,Cundinamarca,Almeidas,...,Arepa Restaurant,Breakfast Spot,Coffee Shop,Restaurant,Wings Joint,Deli / Bodega,Eastern European Restaurant,Diner,Dessert Shop,Department Store
4,25426,5.0815,-73.6076,Machetá,5.080417,-73.608607,City,25,Cundinamarca,Almeidas,...,Plaza,Campground,City,Soccer Field,Dance Studio,Diner,Dessert Shop,Department Store,Deli / Bodega,Convenience Store


In [83]:
# create map of toronto
latitude = 4.598889
longitude = -74.0808
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=8)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Cundinamarca_merged['Codigo_Municipio Latitude'], Cundinamarca_merged['Codigo_Municipio Longitude'], Cundinamarca_merged['Nombre_Municipio'], Cundinamarca_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [84]:
Cundinamarca_merged.groupby('Cluster Labels').count()

,Codigo_Municipio,Codigo_Municipio Latitude,Codigo_Municipio Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Codigo_Depto,Nombre_Depto,Provincia,...,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
Cluster Labels,,,,,,,,,,,,,,,,,,,,,
0,442,442,442,442,442,442,442,442,442,442,...,442,442,442,442,442,442,442,442,442,442
1,45,45,45,45,45,45,45,45,45,45,...,45,45,45,45,45,45,45,45,45,45
2,59,59,59,59,59,59,59,59,59,59,...,59,59,59,59,59,59,59,59,59,59
3,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
4,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
5,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
6,5,5,5,5,5,5,5,5,5,5,...,5,5,5,5,5,5,5,5,5,5
7,6,6,6,6,6,6,6,6,6,6,...,6,6,6,6,6,6,6,6,6,6
8,4,4,4,4,4,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4
